In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Advanced sentiment analysis techniques ready!")

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# 1. Attention Mechanism Implementation
class AttentionLayer(layers.Layer):
    """
    Custom attention layer for RNN-based sentiment analysis
    """
    def __init__(self, units=64, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        # Attention weights
        self.W_a = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True,
            name='attention_weights'
        )
        self.b_a = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True,
            name='attention_bias'
        )
        self.v_a = self.add_weight(
            shape=(self.units, 1),
            initializer='glorot_uniform',
            trainable=True,
            name='attention_context'
        )
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, inputs):
        # inputs shape: (batch_size, time_steps, features)
        
        # Calculate attention scores
        score = tf.nn.tanh(tf.tensordot(inputs, self.W_a, axes=1) + self.b_a)
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.v_a, axes=1), axis=1)
        
        # Apply attention weights
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights
    
    def get_config(self):
        config = super(AttentionLayer, self).get_config()
        config.update({'units': self.units})
        return config

# 2. Hierarchical Attention Network
def create_hierarchical_attention_model(vocab_size, max_length, num_classes, embedding_dim=64):
    """
    Create a hierarchical attention network for document-level sentiment
    """
    inputs = layers.Input(shape=(max_length,))
    
    # Embedding layer
    embedding = layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs)
    
    # Word-level encoder
    word_encoder = layers.Bidirectional(
        layers.LSTM(32, return_sequences=True, dropout=0.2)
    )(embedding)
    
    # Word-level attention
    word_context, word_attention = AttentionLayer(32)(word_encoder)
    
    # Sentence-level processing (simplified for our use case)
    sentence_encoder = layers.Dense(64, activation='relu')(word_context)
    sentence_encoder = layers.Dropout(0.3)(sentence_encoder)
    
    # Final classification
    outputs = layers.Dense(num_classes, activation='softmax')(sentence_encoder)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='HierarchicalAttention')
    return model

# 3. Attention-based LSTM with interpretability
def create_attention_lstm(vocab_size, max_length, num_classes, embedding_dim=64, lstm_units=64):
    """
    Create LSTM with attention mechanism for interpretability
    """
    inputs = layers.Input(shape=(max_length,))
    
    # Embedding
    embedding = layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs)
    
    # LSTM encoder
    lstm_output = layers.LSTM(lstm_units, return_sequences=True, dropout=0.2)(embedding)
    
    # Attention mechanism
    attention_output, attention_weights = AttentionLayer(lstm_units)(lstm_output)
    
    # Classification head
    dense = layers.Dense(32, activation='relu')(attention_output)
    dropout = layers.Dropout(0.3)(dense)
    outputs = layers.Dense(num_classes, activation='softmax')(dropout)
    
    # Create model that also returns attention weights for interpretability
    model = keras.Model(inputs=inputs, outputs=outputs, name='AttentionLSTM')
    
    # Create interpretation model
    interpretation_model = keras.Model(
        inputs=inputs, 
        outputs=[outputs, attention_weights], 
        name='AttentionLSTM_Interpretable'
    )
    
    return model, interpretation_model

# Create sample data for demonstration
def create_extended_sentiment_data():
    """
    Create extended sentiment dataset with more nuanced examples
    """
    positive_texts = [
        "This product is absolutely fantastic! I love everything about it.",
        "Outstanding quality and exceptional customer service. Highly recommended!",
        "Perfect solution for my needs. Couldn't be happier with this purchase.",
        "Amazing value for money. The features are incredible and it works flawlessly.",
        "Exceptional product quality. Fast shipping and excellent packaging.",
        "Love the innovative design and thoughtful features. Simply perfect!",
        "Outstanding performance and reliability. Exceeds all expectations.",
        "Brilliant product with amazing attention to detail. Highly satisfied!"
    ]
    
    negative_texts = [
        "Terrible product quality. Complete waste of money and disappointing experience.",
        "Poor construction and unreliable performance. Avoid this product at all costs.",
        "Awful customer service and defective product. Very frustrated with this purchase.",
        "Overpriced and poor quality. Nothing works as advertised. Total disappointment.",
        "Horrible experience from start to finish. Product broke immediately.",
        "Worst purchase ever made. Cheap materials and terrible design flaws.",
        "Completely unreliable and poorly designed. Save your money and look elsewhere.",
        "Frustrating experience with poor quality and misleading product description."
    ]
    
    neutral_texts = [
        "Average product with standard features. Does what it's supposed to do.",
        "Decent quality for the price range. Nothing extraordinary but functional.",
        "Standard product with typical performance. Meets basic requirements adequately.",
        "Regular quality item with expected functionality. No major complaints.",
        "Ordinary product with basic design. Works as described without surprises.",
        "Acceptable quality and performance. Could be better but serves its purpose.",
        "Standard features and regular build quality. Typical for this price range.",
        "Basic product that meets minimum requirements. Average in most aspects."
    ]
    
    # Create fine-grained sentiment examples
    very_positive = [
        "Absolutely incredible! This is the best product I've ever purchased. Mind-blowing quality!",
        "Phenomenal experience! Everything is perfect and beyond my wildest expectations!"
    ]
    
    very_negative = [
        "Absolutely horrible! This is the worst product imaginable. Complete disaster!",
        "Catastrophically bad! Nothing works and it's a complete waste of money!"
    ]
    
    texts = positive_texts + negative_texts + neutral_texts + very_positive + very_negative
    labels = (['positive'] * len(positive_texts) + 
              ['negative'] * len(negative_texts) + 
              ['neutral'] * len(neutral_texts) + 
              ['very_positive'] * len(very_positive) +
              ['very_negative'] * len(very_negative))
    
    return pd.DataFrame({'text': texts, 'sentiment': labels})

# Create extended dataset
extended_df = create_extended_sentiment_data()
print(f"Extended dataset created with {len(extended_df)} samples")
print(f"Classes: {extended_df['sentiment'].unique()}")
print(f"Class distribution:\n{extended_df['sentiment'].value_counts()}")

# Demonstrate attention mechanism concepts
print(f"\nAttention Mechanism Concepts:")
print("=" * 40)
print("1. Attention helps models focus on relevant words")
print("2. Different words get different importance weights")
print("3. Attention weights sum to 1.0 across sequence")
print("4. Higher weights indicate more important words for prediction")
print("5. Attention provides interpretability to RNN decisions")

# Visualize attention concept
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Simulated attention weights for different examples
examples = [
    "This product is absolutely fantastic and amazing",
    "Poor quality and terrible customer service experience", 
    "Average product with standard basic features only",
    "Incredible quality but expensive for the value"
]

sentiments = ["Positive", "Negative", "Neutral", "Mixed"]
colors = ['green', 'red', 'blue', 'orange']

for i, (example, sentiment, color) in enumerate(zip(examples, sentiments, colors)):
    ax = axes[i//2, i%2]
    words = example.split()
    
    # Simulate attention weights (in real model, these come from attention layer)
    if sentiment == "Positive":
        weights = [0.05, 0.1, 0.05, 0.4, 0.3, 0.05, 0.05]
    elif sentiment == "Negative":
        weights = [0.35, 0.1, 0.05, 0.4, 0.05, 0.025, 0.025]
    elif sentiment == "Neutral":
        weights = [0.15, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15]
    else:  # Mixed
        weights = [0.3, 0.1, 0.05, 0.3, 0.05, 0.1, 0.1]
    
    # Ensure weights sum to 1
    weights = np.array(weights)
    weights = weights / weights.sum()
    
    # Create attention visualization
    bars = ax.bar(range(len(words)), weights, color=color, alpha=0.7)
    ax.set_xticks(range(len(words)))
    ax.set_xticklabels(words, rotation=45, ha='right')
    ax.set_ylabel('Attention Weight')
    ax.set_title(f'{sentiment} Sentiment\n"{example}"')
    ax.set_ylim(0, max(weights) * 1.2)
    
    # Add weight labels
    for bar, weight in zip(bars, weights):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{weight:.2f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

print(f"\nAdvanced sentiment analysis techniques initialized!")
print(f"Ready to implement attention mechanisms and hierarchical models!")
